In [1]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2
from ocean_navigation_simulator.utils.plotting_utils import set_palatino_font, set_palatino_font_plotly
set_palatino_font("ocean_navigation_simulator/package_data/font/Palatino_thin.ttf")
set_palatino_font_plotly("ocean_navigation_simulator/package_data/font/Palatino_thin.ttf")

Layout({
    'font': {'family': 'Palatino', 'size': 21},
    'legend': {'font': {'family': 'Palatino', 'size': 25}},
    'title': {'font': {'family': 'Palatino', 'size': 30}},
    'xaxis': {'tickfont': {'family': 'Palatino', 'size': 13}, 'title': {'font': {'family': 'Palatino', 'size': 25}}},
    'yaxis': {'tickfont': {'family': 'Palatino', 'size': 13}, 'title': {'font': {'family': 'Palatino', 'size': 25}}}
})

In [2]:
wandb_log_name = "CC_HC_GOM_30_platf_coll_100m_flocking_0_5"
wandb_log = False

### Define Filenames

+ Hindcast-Hindcast

In [93]:
# path_to_files = 'generated_media/metrics_csv_results/'
# #file_containing_pb = 'problemsGOM.csv'
# pb_name_hj_naive = '30_platforms_HC_HC_col_100m_hj_naive_1181_fiedler.csv'
# pb_name_reactive = '30_platforms_HC_HC_col_100m_reactive_ctrl_1176_fiedler.csv'
# pb_name_flocking ='30_platforms_HC_HC_col_100m_flocking_softmax_0_5_1170.csv'#30_platforms_HC_HC_col_100m_flocking_0_1_1193_fiedler.csv' #30_platforms_HC_HC_col_100m_flocking_1_1179_fiedler.csv' #'30_platforms_HC_HC_col_100m_flocking_softmax_0_5_1170.csv' #'30_platforms_HC_HC_col_100m_flocking_0_5_1120.csv'#'30_platforms_HC_HC_col_100m_flocking_0_1_1193.csv' #'30_platforms_HC_HC_flocking_0_1_1179.csv'

+ Forecast-Hindcast

In [2]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = '30_platforms_FC_HC_col_100m_hj_naive_1122_fiedler.csv'
pb_name_reactive = '30_platforms_FC_HC_col_100m_reactive_ctrl_1126_fiedler.csv'
pb_name_flocking = '30_platforms_FC_HC_col_100m_flocking_softmax_0_5_1086.csv'#'30_platforms_FC_HC_col_100m_flocking_0_5_1121_fiedler.csv'

### Plot problem distribution

In [71]:
# TODO

### Extract metrics from csv files

In [3]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [4]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [5]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [6]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [7]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  790 corresponds to 80.61224489795919 %


In [126]:
metrics_hj_naive

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mean solver time in seconds,std solver time in seconds,fiedler_mean,fiedler_min,fiedler_max,nb_disconnections
mission_nr,,,,,,,,,,,,,,,,
0,0,1.278164,174,0.000000,0.341171,0.0,9.025048,11,28,0,3.255585,4.600506,2.694495,0.000000e+00,14.488458,76
1,1,0.060185,0,0.533333,0.028881,0.0,5.573025,9,26,0,3.169761,3.937709,3.443675,7.255827e-01,10.938707,0
2,295,1.096065,75,0.600000,0.126338,0.0,1.918189,9,9,0,3.370340,4.504446,0.097998,-1.688715e-15,1.259343,81
3,392,2.542824,0,0.466667,0.064277,0.0,2.055183,15,7,0,3.096417,4.157399,0.507247,-1.553407e-15,4.321330,58
4,491,0.919367,0,0.933333,0.006987,0.0,3.100963,12,8,0,3.169523,3.299697,1.153480,-1.020384e-15,3.573184,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,210,3.671296,0,0.066667,0.282951,0.0,1.315915,10,5,0,3.388427,4.921339,0.126704,-1.547085e-15,0.948655,239
1196,211,0.000000,627,0.833333,0.005845,0.0,3.801888,13,14,0,3.063519,2.976656,1.929220,8.506231e-01,4.654750,0
1197,212,0.194059,634,1.000000,0.000000,0.0,5.368054,12,23,0,3.145285,3.051165,3.888982,4.440892e-16,13.000000,4


+ Drop infeasible missions on HC

In [100]:
metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [8]:
import plotly.express as px
import plotly.graph_objects as go

In [102]:
print("number of missions considered: ", len(metrics_hj_naive))

number of missions considered:  1030


In [ ]:
metrics_hj_naive['fiedler_min'].mean()

In [ ]:
metrics_hj_reactive['fiedler_min'].mean()

In [ ]:
metrics_hj_flocking['fiedler_min'].mean()

In [ ]:
metrics_hj_naive[metrics_hj_naive['Isolated_platform_metric']>0]

## Box Plots

In [9]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Mean Fiedler value 

In [ ]:
fiedler_dict = {"hj_naive": metrics_hj_naive["fiedler_mean"],
                      "hj_reactive": metrics_hj_reactive["fiedler_mean"],
                      "flocking": metrics_hj_flocking["fiedler_mean"]}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Fiedler value over Missions"
)
fig_beta.show()

In [129]:
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )

fiedler_dict = {"MT-HJR-B": np.abs(metrics_hj_naive["fiedler_min"])+1,
                      "MT-HJR-R": np.abs(metrics_hj_reactive["fiedler_min"])+1,
                      "LISIC": np.abs(metrics_hj_flocking["fiedler_min"])+1}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.box(beta_index_avg_df) #,points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    text=beta_index_avg_df.mean().round(2),
    showlegend=True,
    name="mean",
    textposition='middle center',
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
# fig_beta.update_layout(
#     xaxis_title="controllers",
#     yaxis_title="Minimum Fiedler value over Missions",
#     height=800,
#     width=1000
# )
#fig_beta.update_traces(boxpoints=False, selector=dict(type='box'))
fig_beta.update_layout(
    xaxis_title="", #"controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    height=800,
    width=1000,
    plot_bgcolor='white',
    # yaxis=dict(
    #     type='log',
    #     #dtick=0.2,
    #     # tickvals=[1, 10*np.log10(0.5+1), 10*np.log10(1+1), 10*np.log10(2+1)],
    #     # ticktext=['0', '0.5', '1', '2' ],
    #     # range=[0,1],
    #     # showexponent='none'
    # ),
    font=font,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
# Set the subplot titles

fig_beta.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig_beta.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
)
#fig_beta.update_layout(height=800, width=1400,font=font)
fig_beta.write_image("fiedler.svg")
fig_beta.show()

+ Beta Index

In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )


beta_index_avg_dic = {"MT-HJR-B": metrics_hj_naive["Average Beta Index"],
                      "MT-HJR-R": metrics_hj_reactive["Average Beta Index"],
                      "LISIC": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.box(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="",#"controllers",
    yaxis_title="Average Beta Index over Missions",
    font=font,
    plot_bgcolor='white',
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
fig_beta.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig_beta.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
)
fig_beta.update_layout(height=800, width=900,font=font)
fig_beta.write_image("betaIndex.svg")
# Show the plot
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )

idx_list = []
mean_min_dist_dict = {"MT-HJR-B": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "MT-HJR-R": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "LISIC":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}


mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.box(mean_min_dist_df, log_y=False)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="",#"controllers",
    yaxis_title="Minimum centroid distance to target [km]",
    font=font,
    plot_bgcolor='white',
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
fig_min_dist.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig_min_dist.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
)
fig_min_dist.update_layout(height=800, width=900,font=font)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )

opt_ctrl_correction_dict = {"MT-HJR-B": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "MT-HJR-R": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "LISIC": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )

mean_trace = go.Scatter(
    x=opt_ctrl_correction_df.columns,
    y=opt_ctrl_correction_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_opt_ctrl_correction.add_trace(mean_trace)
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title=r'', #"controllers",
    yaxis_title="Max. correction from opt. control [deg.]",
    title = {'text': ""},
    font=font,
    plot_bgcolor='white',
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
# Add custom x and y labels
fig_opt_ctrl_correction.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig_opt_ctrl_correction.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
)
fig_opt_ctrl_correction.update_layout(height=800, width=900,font=font)
fig_opt_ctrl_correction.show()
fig_opt_ctrl_correction.write_image("angleCorrection.svg")

+ Isolated Platform Metric

In [26]:
import plotly.graph_objects as go
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )
ipm_dict= {"MT-HJR-B": metrics_hj_naive["Isolated_platform_metric"],
                      "MT-HJR-R": metrics_hj_reactive["Isolated_platform_metric"],
                      "LISIC":metrics_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    title_text = 'Isolated Platform Metric',
    title_x=0.5,
    xaxis_title="",
    yaxis_title="Isolated Platform Metric (IPM)",
    height=800,
    width=1000,
    plot_bgcolor='white',
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
fig_ipm.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig_ipm.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
    range=[-0.1,7]
)
fig_ipm.update_layout(height=800, width=1200,font=font)
fig_ipm.write_image("ipm.svg")
fig_ipm.show()

In [133]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )

failed_idx_flocking = metrics_hj_flocking["Isolated_platform_metric"]>0

failed_hj_flocking =  metrics_hj_flocking[failed_idx_flocking]
failed_hj_naive = metrics_hj_naive[failed_idx_flocking]
failed_hj_reactive = metrics_hj_reactive[failed_idx_flocking]

ipm_dict_all= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking":metrics_hj_flocking["Isolated_platform_metric"]}

ipm_dict_failed_connect= {"hj_naive": failed_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_hj_flocking["Isolated_platform_metric"]}

all_x_ticks = ['MT-HJR-B']*len(metrics_hj_naive) + ['MT-HJR-R']*len(metrics_hj_reactive) + ['LISIC']*len(metrics_hj_flocking)
all_missions = list(metrics_hj_naive["Isolated_platform_metric"])+list(metrics_hj_reactive["Isolated_platform_metric"])+ list(metrics_hj_flocking["Isolated_platform_metric"])
failed_x_ticks = ['MT-HJR-B']*len(failed_hj_naive) + ['MT-HJR-R']*len(failed_hj_reactive) + ['LISIC']*len(failed_hj_flocking)
failed_missions = list(failed_hj_naive["Isolated_platform_metric"])+list(failed_hj_reactive["Isolated_platform_metric"])+ list(failed_hj_flocking["Isolated_platform_metric"])

#set_palatino_font_plotly("ocean_navigation_simulator/package_data/font/Palatino_thin.ttf")
# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1,horizontal_spacing= 0.0,shared_yaxes=True, shared_xaxes=True)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='LISIC Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='Isolated Platform Metric (IPM)', row=1, col=1)
#fig.update_yaxes(title_text='IPM', row=1, col=2)

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5,  plot_bgcolor='white')
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    #ticks='outside',
    showline=True,
    #linecolor='black',
    linecolor='lightgrey',
    gridcolor='lightgrey',
    range=[-0.1,7]
)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=800, width=1500,font=font)
fig.write_image("ipm_by_mission_type.svg")
# Show the plot
fig.show()


In [115]:
import plotly.graph_objects as go

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Define the box plots
all_box = go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all', line=dict(color=all_missions_color), name='All Missions')
failed_box = go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='Failed Missions Only')

# Define the layout
layout = go.Layout(title='Isolated Platform Metric by Mission Type',
                   xaxis=dict(title='Mission Type'),
                   yaxis=dict(title='IPM', range=[-0.1, 6]),
                   plot_bgcolor='white',
                   xaxis_showgrid=True, yaxis_showgrid=True, 
                   xaxis_gridcolor='lightgrey', yaxis_gridcolor='lightgrey',
                   boxmode='group')

# Define the figure and add the box plots to it
fig = go.Figure(data=[all_box, failed_box], layout=layout)

# Set the dimensions of the figure
fig.update_layout(height=800, width=1200)

# Show the plot
fig.show()



## Get Metrics summaries:

In [103]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "std_beta_index":metrics_file["Average Beta Index"].std(),
    "mean_of_min_fiedler_value":metrics_file["fiedler_min"].abs().mean(),
    "std_of_min_fiedler_value": metrics_file["fiedler_min"].abs().std(),
    "avg_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].mean(),
    "std_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].std(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "std_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].std(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "std_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].std()).km,
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "rate_of_missions_with_disconnection":np.mean(metrics_file["fiedler_min"] < 1e-20), 
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "rate_of_missions_with_collisions": np.mean(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [80]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])
print(summary_hj_naive_df.iloc[0])

ctrl_name                              hj_naive
avg_beta_index                         4.947158
std_beta_index                         1.910102
mean_of_min_fiedler_value              0.392839
std_of_min_fiedler_value               0.636838
avg_isolated_pltf_integral             0.368794
std_isolated_pltf_integral             0.515793
avg_correction_from_opt_ctrl_deg            0.0
std_correction_from_opt_ctrl_deg            0.0
avg_mean_min_dist_to_target_km              0.0
std_mean_min_dist_to_target_km              0.0
nb_missions_with_isolated_pltf              672
rate_of_missions_with_disconnection         0.5
nb_missions_with_collisions                 719
rate_of_missions_with_collisions       0.684762
nb_missions_success                          81
Name: 0, dtype: object


+ HJ Multi-Agent Decentralized Reactive Control

In [81]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
summary_hj_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])
print(summary_hj_reactive.iloc[0])

ctrl_name                              hj_reactive
avg_beta_index                            4.662997
std_beta_index                            1.572072
mean_of_min_fiedler_value                 0.417104
std_of_min_fiedler_value                  0.624242
avg_isolated_pltf_integral                0.195273
std_isolated_pltf_integral                0.340418
avg_correction_from_opt_ctrl_deg         81.824854
std_correction_from_opt_ctrl_deg         25.313072
avg_mean_min_dist_to_target_km            0.142293
std_mean_min_dist_to_target_km            0.559423
nb_missions_with_isolated_pltf                 508
rate_of_missions_with_disconnection       0.451429
nb_missions_with_collisions                      0
rate_of_missions_with_collisions               0.0
nb_missions_success                            425
Name: 0, dtype: object


+ HJ Multi-Agent Flocking

In [104]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
summary_hj_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])
print(summary_hj_flocking.iloc[0])

ctrl_name                              hj_flocking
avg_beta_index                           11.716392
std_beta_index                            1.298495
mean_of_min_fiedler_value                 1.146982
std_of_min_fiedler_value                  1.045507
avg_isolated_pltf_integral                0.046495
std_isolated_pltf_integral                0.199601
avg_correction_from_opt_ctrl_deg        123.097678
std_correction_from_opt_ctrl_deg         21.492193
avg_mean_min_dist_to_target_km            5.897037
std_mean_min_dist_to_target_km           13.114403
nb_missions_with_isolated_pltf                  85
rate_of_missions_with_disconnection       0.099029
nb_missions_with_collisions                      7
rate_of_missions_with_collisions          0.006796
nb_missions_success                            460
Name: 0, dtype: object


## Confusion Matrices Plots

In [105]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates['plotly']["layout"]["font"]["family"] = "Palatino"
font = dict(
                family="Palatino",
                size=30,
                color='black'
            )



def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label], normalize=True)*100
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_traces(texttemplate='%{text:.2f}%', textfont_size=40, text=confusion_matrix.values)
    #fig = px.imshow(confusion_matrix.round(2), text_auto=True)
    fig.update_layout(
        title = {'text': title_name},
        title_x=0.5,
        plot_bgcolor='white',
        height=800, 
        width=1200,
        font=font,
    )
    
    fig.update_coloraxes(
        colorbar_x=0.83,  # Adjust the x-coordinate of the colorbar (0.0 to 1.0)
        colorbar_y=0.5   # Adjust the y-coordinate of the colorbar (0.0 to 1.0)
    )
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey',
    tickfont = dict(
                family="Palatino",
                size=40,
                color='black',
            )
)
    fig.update_yaxes(
        mirror=True,
        #ticks='outside',
        showline=True,
        #linecolor='black',
        linecolor='lightgrey',
        gridcolor='lightgrey',
        tickfont = dict(
            family="Palatino",
            size=40,
            color='black',
        )
        
    )
    fig.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [ ]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["fiedler_min"]<1e-20, 
                        metrics_hj_reactive["fiedler_min"]<1e-20, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")
        

+ Connectivity and Collision maintenance performance

In [108]:
connect_objective_reactive = (metrics_hj_reactive["fiedler_min"]>1e-20) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["fiedler_min"]>1e-20) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'MT-HJR-B', 'MT-HJR-R', title_name= 'Missions w/o collisions and w/ maintained connectivity')
fig_react_maint.write_image("reactive_conf_mat_HC.svg")

+  Missions success counts

In [ ]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [ ]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["fiedler_min"]<1e-20, 
                        metrics_hj_flocking["fiedler_min"]>1e-20, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [110]:
connect_objective_flocking = (metrics_hj_flocking["fiedler_min"]>1e-20) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["fiedler_min"]>1e-20) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'MT-HJR-B', 'LISIC', title_name= 'Missions w/o collisions and w/ maintained connectivity')
fig_flocking_maint.write_image("flocking_conf_mat_HC.svg")

In [ ]:
index_list = metrics_hj_flocking.index[(connect_objective_reactive==True) & (connect_objective_flocking==False)].tolist()


In [ ]:
index_list

In [ ]:
metrics_hj_reactive.loc[285]

+  Missions success counts

In [ ]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

In [111]:
connect_objective_flocking = (metrics_hj_flocking["fiedler_min"]>1e-20) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_reactive = (metrics_hj_reactive["fiedler_min"]>1e-20) & (metrics_hj_reactive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_reactive , connect_objective_flocking , 
    'MT-HJR-R', 'LISIC', title_name= 'Missions w/o collisions and w/ maintained connectivity')
fig_flocking_maint.write_image("react_vs_flock_conf_mat_HC.svg")

# Statistical Testing

Welch's t-test, or unequal variances t-test, is a two-sample location test which is used to test the (null) hypothesis that two populations have equal means

Yes, in this case, a Welch's t-test would be a better choice than a two-sample t-test assuming equal variances. The reason is that the variance of the beta values may not be equal between the two controllers. Welch's t-test is designed to handle this situation by adjusting the degrees of freedom used in the t-test, which provides a more accurate p-value.

The code for performing the Welch's t-test would be very similar to the code for the two-sample t-test, but with the equal_var parameter set to False, which is the default value for ttest_ind:

In [ ]:
import numpy as np
from scipy.stats import ttest_ind

# reactive_X = list(metrics_hj_reactive['Average Beta Index']) 
# flocking_X = list(metrics_hj_flocking['Average Beta Index'])

# reactive_X = list(metrics_hj_reactive['Isolated_platform_metric']) 
# flocking_X= list(metrics_hj_flocking['Isolated_platform_metric'])
# reactive_X = list(abs(metrics_hj_reactive['fiedler_min']))
# flocking_X = list(abs(metrics_hj_flocking['fiedler_min']))

reactive_X = list(Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km)
flocking_X = list(Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km)

reactive_X = list(metrics_hj_reactive['Mean maximum correction from optimal control degrees']) 
flocking_X = list(metrics_hj_flocking['Mean maximum correction from optimal control degrees'])

 #Compute the mean and standard deviation of beta values for each controller
reactive_mean = np.mean(reactive_X)
flocking_mean = np.mean(flocking_X)
reactive_std = np.std(reactive_X, ddof=1)
flocking_std = np.std(flocking_X, ddof=1)

# Compute the t-statistic and p-value using a two-sample t-test assuming unequal variances
t_stat, p_value = ttest_ind(reactive_X, flocking_X, equal_var=False) # perform Welch's t-test since the variances are unequal

# Print the results
print(f"Reactive controller mean: {reactive_mean:.3f}, std: {reactive_std:.3f}")
print(f"Flocking controller mean: {flocking_mean:.3f}, std: {flocking_std:.3f}")
print(f"Two-sample t-test results: t-statistic={t_stat:.3f}, p-value={p_value}")

if p_value < 0.05:
    print("The difference between the two controllers is statistically significant.")
else:
    print("There is no statistically significant difference between the two controllers.")


A one-sample location test of whether the mean of a population has a value specified in a null hypothesis.
However: A one-sample t-test would not be appropriate for comparing the performance of two controllers, as it is designed to test whether a single sample comes from a population with a specified mean.

Since you have 1000 samples of the beta value for each controller, a more appropriate test would be a two-sample t-test. This test is designed to compare the means of two independent samples and would allow you to test whether the difference in means between the reactive and flocking controllers is statistically significant.

In [ ]:
import numpy as np
from scipy.stats import ttest_1samp

reactive_X = list(metrics_hj_reactive['Average Beta Index']) 

flocking_X = list(metrics_hj_flocking['Average Beta Index'])
flocking_mean = np.mean(flocking_X)  # define flocking_mean

# Compute the mean and standard deviation of beta values for the reactive controller
reactive_mean = np.mean(reactive_X)
reactive_std = np.std(reactive_X, ddof=1)

# Compute the t-statistic and p-value using a one-sample t-test assuming that flocking performs better than reactive
t_stat, p_value = ttest_1samp(reactive_X, flocking_mean) #

# Print the results
print(f"Reactive controller mean: {reactive_mean:.3f}, std: {reactive_std:.3f}")
print(f"Flocking controller mean: {flocking_mean:.3f}")
print(f"One-sample t-test results: t-statistic={t_stat:.3f}, p-value={p_value}")

if p_value < 0.05:
    print("The flocking controller performs statistically significantly better than the reactive controller.")
else:
    print("There is no statistically significant difference between the two controllers.")


+ Collisions: no statistical significance ?

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= len(metrics_hj_naive)
N_success_reactive = len(metrics_hj_reactive[metrics_hj_reactive['Number_of_collision']==0])
N_success_flocking = len(metrics_hj_flocking[metrics_hj_flocking['Number_of_collision']==0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_success_flocking, N_success_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="larger") #reject H0 if pval < 0.05 in favor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= len(metrics_hj_naive)
N_failed_reactive = len(metrics_hj_reactive[metrics_hj_reactive['Number_of_collision']>0])
N_failed_flocking = len(metrics_hj_flocking[metrics_hj_flocking['Number_of_collision']>0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_failed_flocking, N_failed_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="smaller") #reject H0 if pval < 0.05 in favor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

+ Communication: statistical significance ?

H0 is that there is no significant difference between the success rate of the reactive and the flocking methods in favor of the alternative hypothesis that the proportion of success for the flocking method is greater than the proportion of success for the flocking. Can reject H0 is p-value <= 0.05

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= len(metrics_hj_naive)
N_success_reactive = len(metrics_hj_reactive[metrics_hj_reactive['fiedler_min']>0])
N_success_flocking = len(metrics_hj_flocking[metrics_hj_flocking['fiedler_min']>0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_success_flocking, N_success_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="larger") #reject H0 if pval < 0.05 in favor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= 1000
N_failed_reactive = 580#448#len(metrics_hj_reactive[metrics_hj_reactive['Isolated_platform_metric']>0])
N_failed_flocking = 99#99#len(metrics_hj_flocking[metrics_hj_flocking['Isolated_platform_metric']>0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_failed_flocking, N_failed_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="smaller") #reject H0 if pval < 0.05 in favor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

## Log Data in WandB

In [ ]:
import wandb

In [ ]:
# Log metrics in WandB
if wandb_log:
    os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

    wandb.init(
        # Set the project where this run will be logged
        project="Master_Thesis_Run_Summaries",
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=wandb_log_name,
        # Track hyperparameters and run metadata
        config={
            "Number of infeasible missions": len(non_feas_idx),
            "Percentage of infeasible missions": ratio_non_feas_miss,
            "Number of missions feasible": len(metrics_hj_naive),
            "Ratio failing missions with HJ naive in terms of connectivity and collisions": 
            len(metrics_hj_naive.index[(connect_objective_hj_naive==False)].tolist())/len(metrics_hj_naive),
        },
        entity="nhoischen",
    )
    wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
                commit=False)
    wandb.log({'Beta Index': fig_beta,
            'Time-Integral-over-Isolated-Platforms': fig_ipm,
            'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
            'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
                },
            commit=False)
    wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
            'Missions with collisions hj-flocking': fig_conf_flocking_coll,
            'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
            'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
            'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
            'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
            'Succesfull Missions hj-reactive': fig_react_success,
            'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
    wandb.finish()